In [15]:
#required modules
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [10]:
#paths
image_dir = "Dubai_images"
csv_path = "labels_final.csv"

#loading the CSV
df = pd.read_csv(csv_path)

#Lists to store data
images = []
labels = []

In [11]:
for _, row in df.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # grayscale
    if img is not None:
        img = cv2.resize(img, (128, 64))               # resizing
        img = img / 255.0                              # normalize
        images.append(img)
        labels.append(row['plate_text'])

In [12]:
#loading all the images
X = np.array(images).reshape(-1, 64, 128, 1)
y = np.array(labels)

print("Loaded", len(X), "images with shape", X.shape)
print("Sample label:", y[0])

Loaded 240 images with shape (240, 64, 128, 1)
Sample label: P 12034


In [13]:
#ensuring all images have loaded
missing = []

for _, row in df.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        missing.append(row['filename'])

print("Missing or unreadable images:", missing)
print("Total missing:", len(missing))

Missing or unreadable images: []
Total missing: 0


In [14]:
#encode lables
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

#one-hot encode labels for categorical classification
y_encoded = to_categorical(encoded_labels)

print("Total classes:", y_encoded.shape[1])
print("Sample encoded label:", y_encoded[0])

Total classes: 219
Sample encoded label: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [16]:
#defining the CNN model
model = Sequential()

#first convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 128, 1)))
model.add(MaxPooling2D(2, 2))

#second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

#flatten + dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_encoded.shape[1], activation='softmax'))  #output layer

#compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#show model summary
model.summary()

C:\Users\REIA\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 126, 32)         │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 26880)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,440,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 219)                 │          28,251 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,487,835 (13.31 MB)

 Trainable params: 3,487,835 (13.31 MB)

 Non-trainable params: 0 (0.00 B)